In [6]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 
# import talib as ta
import math
# from numba import jit

from importlib import reload 
from qflib import basic
reload(basic)
global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

In [7]:
# 指标库读出基本数据 
xl_01dgree = 0.001745 # 0.1度 
xl_02dgree = 0.003490 # 0.2度 
xl_03dgree = 0.005230 # 0.3度
xl_dgree = xl_02dgree
print( str(xl_dgree) )

sql = "SELECT * \
from day_zhibiao \
where \
jx_days_ud60 > 20 and jx_days_ud60 < 40 and \
jx_xl_60>-"+str(xl_dgree)+" and jx_xl_20>0 and \
lj_fl_5_20>1 "
print(sql)
df = pd.read_sql_query(sql, conn)
# df.to_sql('day_zhibiao_qf60', conn, index=False, if_exists='append', chunksize=2000) 
# sql = 'select * from day_zhibiao_qf60'
# df = pd.read_sql_query(sql, conn)
df.shape

0.00349
SELECT * from day_zhibiao where jx_days_ud60 > 20 and jx_days_ud60 < 40 and jx_xl_60>-0.00349 and jx_xl_20>0 and lj_fl_5_20>1 


(165293, 76)

In [8]:
""" 数据抽查 """
# bins = [-float('inf'), 0, 2, 5, 10, float('inf')] 
# labels = ['<0', '0-2', '2-5', '5-10', '>10']
# df['sz_max_20_bin'] = pd.cut(df['sz_max_20'], bins=bins, labels=labels)
# df['sz_max_20_bin'].value_counts()
bins = [-float('inf'), 0, 10, float('inf')] 
labels = ['=0', '0-10', '>10']
df['sz_max_20_bin'] = pd.cut(df['sz_max_20'], bins=bins, labels=labels)
df['sz_max_20_bin'].value_counts()

0-10    95821
>10     46832
=0      22640
Name: sz_max_20_bin, dtype: int64

In [9]:
""" 抽查验证 """
# df[ df['sz_max_20']>10  ]

' 抽查验证 '

In [10]:
# """ 字段优化处理 """
# # 斜率改为角度， 再乘100， 扩大影响
# df['jx_dg_120'] = df['jx_xl_120'].apply(np.arctan).apply(np.degrees) * 100
# df['jx_dg_60'] = df['jx_xl_60'].apply(np.arctan).apply(np.degrees)  * 100
# df['jx_dg_20'] = df['jx_xl_20'].apply(np.arctan).apply(np.degrees)  * 100
# df['jx_dg_10'] = df['jx_xl_10'].apply(np.arctan).apply(np.degrees)  * 100
# df['jx_dg_5'] = df['jx_xl_5'].apply(np.arctan).apply(np.degrees)  * 100

# # 放量正确化、乘100，成百分比
# df['pct_fl_1_3']  = (df['lj_fl_1_3'] - 1) * 3/2  * 100
# df['pct_fl_3_10']  = (df['lj_fl_3_10'] - 1) * 3/2  * 100
# df['pct_fl_5_20']  = (df['lj_fl_5_20'] - 1) *20/15 * 100
# df['pct_fl_5_60']  = (df['lj_fl_5_60'] - 1) *20/15 * 100
# df['pct_fl_20_60'] = (df['lj_fl_20_60'] -1) *60/40 * 100
# # 站上20日线
# df['zs_20'] = (df['jx_zs_20'] > 0).astype(int)
# # 查看测试结果
# df[ ['jx_dg_60', 'jx_xl_60', 'pct_fl_5_20','lj_fl_5_20', 'zs_20'] ].head(10)

In [11]:
# 涨幅阶梯化

def jieti_10(df):
    # rate = 0, 1, 2, 3, 4, 5
    if df['sz_max_10'] == 0: rate = 0        
    if df['sz_max_10']>0 and df['sz_max_10']<=3: rate = 1        
    if df['sz_max_10']>3 and df['sz_max_10']<=6: rate = 2        
    if df['sz_max_10']>6 and df['sz_max_10']<=10: rate = 3        
    if df['sz_max_10']>10 and df['sz_max_10']<=20: rate = 4        
    if df['sz_max_10']>20: rate = 5        
    return rate
def jieti_20(df):
    # rate = 0, 1, 2
    if df['sz_max_20'] == 0: rate = 0        
    if df['sz_max_20']>0 and df['sz_max_10']<=10: rate = 1        
    # if df['sz_max_20']>10 and df['sz_max_10']<=20: rate = 2        
    if df['sz_max_20']>10: rate = 2        
    return rate

df['sz_jt_10'] = df.apply(jieti_10, axis=1) 
df['sz_jt_20'] = df.apply(jieti_20, axis=1) 

In [12]:
df[ ['sz_max_20', 'sz_jt_20']].head(20)

,sz_max_20,sz_jt_20
0,3.65759,1
1,2.46154,1
2,3.01624,1
3,1.62822,1
4,0.00000,0
5,1.24526,1
6,1.30011,1
7,3.31492,1
8,0.00000,0
9,0.00000,0


In [13]:
""" 形成数据集 """

# 确定训练的主要目标s：
df['test00'] = df['sz_max_20'] > 10
df['test5_8'] = df['sz_max_5'] > 8
df['test10_10'] = df['sz_max_10'] > 10
df['test20_10'] = df['sz_max_20'] > 10
df['test20_20'] = df['sz_max_20'] > 20
df['test20_30'] = df['sz_max_20'] > 30
# df['class0'] = df['test00']
df['class0'] = df['sz_jt_20']
# 确定输出列  
col_list_ds = ['ts_code', 'trade_date', 'pct_change',  \
    'diff', 'dea', 'bar',\
    'jx_days_ud60',\
    'jx_dg_120', 'jx_dg_60', \
    'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_20_60', \
    'test00', 'test5_8', 'test10_10', 'test20_10',  'test20_20', 'test20_30', \
    'sz_jt_10','sz_jt_20', 'class0' ]
df_ds = df[col_list_ds].copy()
# print(df_ds.head(2))
# df[ df['sz_max_10']>10 ].shape, df[ df['sz_max_20']>20 ].shape, df[ df['sz_max_20']>30 ].shape
df_ds[ df_ds['test00'] ].shape, df_ds[ df_ds['test5_8'] ].shape, df_ds[ df_ds['test10_10'] ].shape, df_ds[df_ds['test20_10']].shape, df_ds[df_ds['test20_20']].shape, df_ds[ df_ds['test20_30'] ].shape

((46832, 29), (17004, 29), (24638, 29), (46832, 29), (15764, 29), (6573, 29))

In [14]:
df_ds[ df_ds['test00'] ]

,ts_code,trade_date,pct_change,diff,dea,bar,jx_days_ud60,jx_dg_120,jx_dg_60,jx_dg_20,...,pct_fl_20_60,test00,test5_8,test10_10,test20_10,test20_20,test20_30,sz_jt_10,sz_jt_20,class0
10,000001.SZ,20190115,1.2859,-0.013003,-0.130142,0.234278,28,7.173730,-4.814930,2.062700,...,-31.7441,True,False,False,True,False,False,3,2,2
31,000002.SZ,20220902,-0.2375,-0.224587,-0.324692,0.200210,37,0.079105,-5.740190,5.926910,...,-11.3821,True,True,True,True,False,False,4,2,2
33,000002.SZ,20220831,4.6570,-0.343047,-0.367997,0.049900,35,-2.742170,-4.918370,9.965080,...,-23.0370,True,False,True,True,False,False,4,2,2
34,000002.SZ,20220621,0.0000,-0.076781,-0.218459,0.283356,21,-2.350580,9.041910,3.669840,...,-49.8885,True,False,True,True,False,False,4,2,2
41,000002.SZ,20190626,-0.3228,0.041086,-0.046900,0.175973,38,6.705630,-3.845090,1.766240,...,-41.4383,True,False,False,True,False,False,3,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165241,600199.SH,20190819,1.7915,-0.138959,-0.210179,0.142439,24,5.960100,-2.773410,5.682690,...,-39.5781,True,False,False,True,False,False,3,2,2
165242,600199.SH,20190625,-1.4815,-0.012452,-0.094965,0.165026,27,14.548900,-13.498200,5.391270,...,-73.5340,True,True,True,True,False,False,4,2,2
165243,600199.SH,20190624,-0.8811,-0.031192,-0.115594,0.168805,26,15.517000,-11.130400,13.940600,...,-76.9888,True,True,True,True,False,False,4,2,2
165258,600199.SH,20150316,1.5900,0.020114,-0.055410,0.151048,26,8.807590,-0.436546,21.732000,...,-36.8418,True,False,False,True,False,False,3,2,2


In [17]:
""" 输出到mysql """
# 删除旧数据
sql = "truncate table ds_qf60"
basic.exec_sql(conn, sql)
# 输出新数据
df_ds.to_sql('ds_qf60', conn, index=False, if_exists='append', chunksize=2000) 

165293

In [18]:
df_ds

,ts_code,trade_date,pct_change,diff,dea,bar,jx_days_ud60,jx_dg_120,jx_dg_60,jx_dg_20,...,pct_fl_20_60,test00,test5_8,test10_10,test20_10,test20_20,test20_30,sz_jt_10,sz_jt_20,class0
0,000001.SZ,20230419,-1.1538,-0.108607,-0.190289,0.163366,32,8.16319,-15.010000,3.387080,...,-20.23010,False,False,False,False,False,False,2,1,1
1,000001.SZ,20230418,0.5414,-0.127556,-0.210710,0.166309,31,7.00177,-14.688300,9.715350,...,-21.35470,False,False,False,False,False,False,0,1,1
2,000001.SZ,20230417,1.8913,-0.165159,-0.231499,0.132679,30,6.34968,-14.228100,4.073800,...,-24.00660,False,False,False,False,False,False,1,1,1
3,000001.SZ,20220615,4.3909,-0.171259,-0.239591,0.136663,30,-10.72400,6.763570,0.599788,...,3.86983,False,False,False,False,False,False,0,1,1
4,000001.SZ,20220121,0.1154,-0.182113,-0.249464,0.134700,30,-1.07083,-12.851900,0.507476,...,25.59640,False,False,False,False,False,False,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165288,600201.SH,20230515,-3.6251,-0.066965,-0.131964,0.129998,23,6.71357,-11.829100,0.265849,...,-3.62789,False,False,False,False,False,False,0,0,0
165289,600201.SH,20230512,4.1436,-0.079844,-0.148213,0.136739,22,10.40010,-7.082810,7.178250,...,-9.88584,False,False,False,False,False,False,0,0,0
165290,600201.SH,20220905,0.0000,0.011637,-0.018744,0.060763,32,-19.30980,-0.434874,1.002780,...,-52.85400,False,False,False,False,False,False,1,1,1
165291,600201.SH,20220902,0.9281,0.003796,-0.026340,0.060272,31,-17.96200,1.521940,6.686400,...,-53.25980,False,False,False,False,False,False,1,1,1
